In [1]:
import numpy as np
import pandas as pd
import os
import json
import h5py

from sklearn.model_selection import train_test_split

os.environ["KERAS_BACKEND"] = "tensorflow"

import tensorflow as tf

import keras
import keras.backend as K

from tqdm import tqdm

2025-07-16 11:28:41.826035: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752665322.021674      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752665322.080269      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# 4. Regression metrics

In [2]:
@tf.function
def rmse(y_true, y_pred):
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred, tf.float32)
    
    y_true = tf.reshape(y_true, [-1, 1])
    y_pred = tf.reshape(y_pred, [-1, 1])
    
    return tf.sqrt(tf.reduce_mean(tf.square(y_true - y_pred)))

@tf.function
def eva(y_true, y_pred):
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred, tf.float32)
    
    y_true = tf.reshape(y_true, [-1, 1])
    y_pred = tf.reshape(y_pred, [-1, 1])

    numerator = tf.math.reduce_variance(y_true - y_pred)
    denominator = tf.math.reduce_variance(y_true) + K.epsilon()
    
    return 1.0 - numerator / denominator

@tf.function
def r2_score(y_true, y_pred):
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred, tf.float32)
    
    y_true = tf.reshape(y_true, [-1, 1])
    y_pred = tf.reshape(y_pred, [-1, 1])
    
    ss_res = tf.reduce_sum(tf.square(y_true - y_pred))
    ss_tot = tf.reduce_sum(tf.square(y_true - tf.reduce_mean(y_true)))
    
    return 1 - (ss_res / (ss_tot + K.epsilon()))

# 4. ResNet 8/18/34 construction

In [3]:
def resnet_block(x, filters, stride=1, use_projection=False):
    shortcut = x

    x = keras.layers.Conv2D(filters, 3, strides=stride, padding='same', use_bias=False)(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.ReLU()(x)

    x = keras.layers.Conv2D(filters, 3, strides=1, padding='same', use_bias=False)(x)
    x = keras.layers.BatchNormalization()(x)

    if use_projection:
        shortcut = keras.layers.Conv2D(filters, 1, strides=stride, use_bias=False)(shortcut)
        shortcut = keras.layers.BatchNormalization()(shortcut)

    x = keras.layers.Add()([x, shortcut])
    x = keras.layers.ReLU()(x)
    return x

In [4]:
def ResNet8_DroNet(input_shape=(200, 200, 1), dropout_rate=0.3):
    inputs = keras.Input(shape=input_shape)

    x = keras.layers.Conv2D(32, 5, strides=2, padding='same', use_bias=False)(inputs)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.ReLU()(x)

    # Residual Blocks
    x = resnet_block(x, 32)
    x = resnet_block(x, 32)

    x = resnet_block(x, 64, stride=2, use_projection=True)
    x = resnet_block(x, 64)

    x = resnet_block(x, 128, stride=2, use_projection=True)
    x = resnet_block(x, 128)

    return keras.models.Model(inputs, x, name="ResNet-8")

In [5]:
def ResNet18(input_shape=(224, 224, 3)):
    inputs = keras.Input(shape=input_shape)
    
    x = keras.layers.Conv2D(64, 7, strides=2, padding='same', use_bias=False)(inputs)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.ReLU()(x)
    x = keras.layers.MaxPooling2D(3, strides=2, padding='same')(x)

    # Conv2_x
    x = resnet_block(x, 64)
    x = resnet_block(x, 64)

    # Conv3_x
    x = resnet_block(x, 128, stride=2, use_projection=True)
    x = resnet_block(x, 128)

    # Conv4_x
    x = resnet_block(x, 256, stride=2, use_projection=True)
    x = resnet_block(x, 256)

    # Conv5_x
    x = resnet_block(x, 512, stride=2, use_projection=True)
    x = resnet_block(x, 512)

    return keras.models.Model(inputs, x, name="ResNet-18")

In [6]:
def ResNet34(input_shape=(224, 224, 3)):
    inputs = keras.Input(shape=input_shape)
    
    x = keras.layers.Conv2D(64, 7, strides=2, padding='same', use_bias=False)(inputs)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.ReLU()(x)
    x = keras.layers.MaxPooling2D(3, strides=2, padding='same')(x)

    # Conv2_x (3 blocks)
    for _ in range(3):
        x = resnet_block(x, 64)

    # Conv3_x (4 blocks)
    x = resnet_block(x, 128, stride=2, use_projection=True)
    for _ in range(3):
        x = resnet_block(x, 128)

    # Conv4_x (6 blocks)
    x = resnet_block(x, 256, stride=2, use_projection=True)
    for _ in range(5):
        x = resnet_block(x, 256)

    # Conv5_x (3 blocks)
    x = resnet_block(x, 512, stride=2, use_projection=True)
    for _ in range(2):
        x = resnet_block(x, 512)

    return keras.models.Model(inputs, x, name="ResNet-34")

# Grid search configurations

In [7]:
from itertools import product

configurations = list(product([1e-2, 1e-3], [32, 64]))

# Models descriptions

In [8]:
models_descriptions = [
    {
        "name": "ResNet-8-base",
        "backbone": {
            "name": "ResNet-8",
            "weights": "dronet",
        },
        "dropout_rate": 0.5,
        "input": {
            "shape": [200, 200, 1],
            "grayscale": True,
        },
        "head_description": [],
    },
    {
        "name": "ResNet-8-d32",
        "backbone": {
            "name": "ResNet-8",
            "weights": "dronet",
        },
        "dropout_rate": 0.5,
        "input": {
            "shape": [200, 200, 1],
            "grayscale": True,
        },
        "head_description": [(32, False, False)],
    },
    {
        "name": "ResNet-8-d64",
        "backbone": {
            "name": "ResNet-8",
            "weights": "dronet",
        },
        "dropout_rate": 0.5,
        "input": {
            "shape": [200, 200, 1],
            "grayscale": True,
        },
        "head_description": [(64, False, False)],
    },
    {
        "name": "ResNet-8-d128",
        "backbone": {
            "name": "ResNet-8",
            "weights": "dronet",
        },
        "dropout_rate": 0.5,
        "input": {
            "shape": [200, 200, 1],
            "grayscale": True,
        },
        "head_description": [(128, False, False)],
    },
    {
        "name": "MobileNet",
        "backbone": {
            "name": "MobileNet",
            "weights": "imagenet",
        },
        "input": {
            "shape": [224, 224, 3],
        },
    },
    {
        "name": "EfficientNetB0",
        "backbone": {
            "name": "EfficientNetB0",
            "weights": "imagenet",
        },
        "dropout_rate": 0.2,
        "input": {
            "shape": [224, 224, 3],
        },
    },
    {
        "name": "ResNet-50",
        "backbone": {
            "name": "ResNet-50",
            "weights": "imagenet",
        },
        "input": {
            "shape": [224, 224, 3],
        },
    },
    {
        "name": "ResNet-50V2",
        "backbone": {
            "name": "ResNet-50V2",
            "weights": "imagenet",
        },
        "input": {
            "shape": [224, 224, 3],
        },
    },
]

# Format verification
for model_description in models_descriptions:
    if "backbone" not in model_description:
        raise KeyError("backbone not specified at:\n" + str(model_description))

    if "name" not in model_description["backbone"]:
        raise KeyError("backbone.name not specified at:\n" + str(model_description))

    if "weights" not in model_description["backbone"]:
        model_description["backbone"]["weights"] = None

    if "freeze" not in model_description["backbone"]:
        model_description["backbone"]["freeze"] = True
    
    if "name" not in model_description:
        model_description["name"] = model_description["backbone"]["name"]

    if "input" not in model_description:
        raise KeyError("input not specified at:\n" + str(model_description))

    if "shape" not in model_description["input"]:
        raise KeyError("input.shape not specified at:\n" + str(model_description))
    
    if "grayscale" not in model_description["input"]:
        model_description["input"]["grayscale"] = False

    if "head_description" not in model_description:
        model_description["head_description"] = []

# Load dataset and build pipeline

In [9]:
def load_samples_dataframe(path: str):
    images_folder = os.path.join(path, "images")

    samples_df = pd.read_csv(
        os.path.join(path, "market_dataset_xy.txt"),
        sep=" ",
        header=None,
        names=["file path", "_", "datetime", "vel_y", "vel_x"]
    )

    samples_df["file path"] = (
        samples_df["file path"]
        .apply(lambda image_name: os.path.join(path, "images", image_name))
    )
    
    return samples_df

In [10]:
def build_train_val_test_datasets(samples_dataframe, input_description: dict, seed=42):
    resolution = input_description["shape"][:2]
    channels = input_description["shape"][2]

    df_train, df_temp = train_test_split(samples_dataframe, test_size=0.4, random_state=seed)
    df_val, df_test = train_test_split(df_temp, test_size=0.5, random_state=seed)

    if not input_description["grayscale"]:
        mode = "color"
    elif channels == 1:
        mode = "grayscale1"
    elif channels == 3:
        mode = "grayscale3"

    @tf.function
    def process_sample(path, label):
        image = tf.io.read_file(path)

        if mode == "color":
            image = tf.image.decode_jpeg(image, channels=3)
    
        elif mode == "grayscale1":
            image = tf.image.decode_jpeg(image, channels=1)
    
        elif mode == "grayscale3":
            image = tf.image.decode_jpeg(image, channels=3)
            image = tf.image.rgb_to_grayscale(image)
            image = tf.image.grayscale_to_rgb(image)

        image = tf.image.resize(image, resolution)
        image.set_shape(input_description["shape"])

        return image, label

    def df_to_tf_dataset(df):
        paths = tf.constant(df["file path"].tolist())
        vel_x = tf.constant(df["vel_x"].astype("float32").tolist())
        vel_y = tf.constant(df["vel_y"].astype("float32").tolist())
    
        ds = tf.data.Dataset.from_tensor_slices((paths, {"vel_x": vel_x, "vel_y": vel_y}))

        ds = ds.map(process_sample, num_parallel_calls=tf.data.AUTOTUNE)
        ds = ds.cache()
    
        return ds

    return {
        "train": df_to_tf_dataset(df_train),
        "val":   df_to_tf_dataset(df_val),
        "test":  df_to_tf_dataset(df_test),
    }

In [11]:
def get_augment_function(gen):
    @tf.function
    def augment(image, label):
        delta = gen.uniform([], minval=-0.2, maxval=0.2)
        image = tf.image.adjust_brightness(image, delta * 255.0)
        
        contrast_factor = gen.uniform([], minval=0.8, maxval=1.2)
        image = tf.image.adjust_contrast(image, contrast_factor)
        
        noise = gen.normal(tf.shape(image), mean=0.0, stddev=0.025 * 255.0)
        image = image + noise
        image = tf.clip_by_value(image, 0.0, 255.0)
        
        do_flip = gen.uniform([], minval=0, maxval=1) > 0.5
        image = tf.cond(do_flip, lambda: tf.image.flip_left_right(image), lambda: image)
        
        vel_x = tf.cond(do_flip, lambda: -label["vel_x"], lambda: label["vel_x"])
        vel_y = label["vel_y"]
        
        return image, {"vel_x": vel_x, "vel_y": vel_y}

    return augment

In [12]:
def instantiate_backbone(backbone_name, weights=None):
    if backbone_name == "ResNet-8":
        if not weights:
            return ResNet8_DroNet(input_shape=(200, 200, 1))
        elif weights == "dronet":
            model = keras.models.load_model("/kaggle/input/marketplace-navigation-dataset/model.keras", compile=True)
            model.name = "ResNet-8"
            return model
        else:
            raise ValueError("No", weights, "weights found for", backbone_name)
    
    if backbone_name == "ResNet-18":
        if weights is not None:
            raise ValueError("No", weights, "weights found for", backbone_name)
        return ResNet18(input_shape=(224, 224, 3))
        
    if backbone_name == "ResNet-34":
        if weights is not None:
            raise ValueError("No", weights, "weights found for", backbone_name)
        return ResNet34(input_shape=(224, 224, 3))

    if backbone_name == "ResNet-50":
        assert not weights or weights in ["imagenet"], "No " + weights + " weights found for " + backbone_name
        return keras.applications.ResNet50(
            name="ResNet-50",
            include_top=False,
            input_shape=(224, 224, 3),
            weights=weights,
            pooling=None,
        )

    if backbone_name == "ResNet-50V2":
        assert not weights or weights in ["imagenet"], "No " + weights + " weights found for " + backbone_name
        return keras.applications.ResNet50V2(
            name="ResNet-50V2",
            include_top=False,
            input_shape=(224, 224, 3),
            weights=weights,
            pooling=None,
        )

    if backbone_name == "MobileNet":
        assert not weights or weights in ["imagenet"], "No " + weights + " weights found for " + backbone_name
        return keras.applications.MobileNet(
            name="MobileNet",
            include_top=False,
            input_shape=(224, 224, 3),
            weights=weights,
            pooling=None,
            alpha=1.0,
        )

    if backbone_name == "MobileNetV2":
        assert not weights or weights in ["imagenet"], "No " + weights + " weights found for " + backbone_name
        return keras.applications.MobileNetV2(
            name="MobileNetV2",
            include_top=False,
            input_shape=(224, 224, 3),
            weights=weights,
            pooling=None,
            alpha=1.0,
        )

    if backbone_name == "EfficientNetB0":
        assert not weights or weights in ["imagenet"], "No " + weights + " weights found for " + backbone_name
        # As of 04/07/2025, setting name="EfficientNetB0" causes a
        # HTTP error for reasons that neither I or ChatGPT can comprehend
        # return keras.applications.EfficientNetB0(
        #     include_top=False,
        #     name="EfficientNetB0",
        #     input_shape=(224, 224, 3),
        #     weights=weights,
        #     pooling=None,
        # )

        # This should work
        model = keras.applications.EfficientNetB0(
            include_top=False,
            input_shape=(224, 224, 3),
            weights=weights,
            pooling=None,
        )
        model.name = "EfficientNetB0"
        return model
    
    raise ValueError(f"Backbone {backbone_name} not implemented.")

In [13]:
def build_model(model_description: dict):
    input_shape = model_description["input"]["shape"]
    input_shape = tuple(input_shape)

    backbone_name = model_description["backbone"]["name"]
    backbone_weights = model_description["backbone"]["weights"]
    freeze_backbone = model_description["backbone"]["freeze"]

    model_name = model_description.get("name", backbone_name)
    dropout_rate = model_description.get("dropout_rate", None)

    PREPROCESS = {
        "ResNet-8":        lambda x: x / 255.0,
        "ResNet-18":       lambda x: x / 255.0,
        "ResNet-34":       lambda x: x / 255.0,
        "MobileNet":       keras.applications.mobilenet.preprocess_input,
        "MobileNetV2":     keras.applications.mobilenet_v2.preprocess_input,
        "ResNet-50":       keras.applications.resnet.preprocess_input,
        "ResNet-50V2":     keras.applications.resnet_v2.preprocess_input,
        "EfficientNetB0":  keras.applications.efficientnet.preprocess_input,
    }

    backbone = instantiate_backbone(backbone_name, weights=backbone_weights)
    backbone.trainable = not freeze_backbone

    inputs = keras.layers.Input(input_shape)
    x = PREPROCESS.get(backbone_name, lambda x: x)(inputs)
    x = backbone(x)

    if backbone_name == "ResNet-8":
        x = keras.layers.Flatten(name="flatten")(x)
        x = keras.layers.ReLU(name="flatten_ReLU")(x)
    else:
        x = keras.layers.GlobalAveragePooling2D(name="global_pool")(x)

    if dropout_rate:
        x = keras.layers.Dropout(dropout_rate, name="head_dropout")(x)

    for i, (nodes, use_batch_normalization, use_dropout) in enumerate(model_description["head_description"]):
        use_bias = not use_batch_normalization
        
        x = keras.layers.Dense(nodes, use_bias=use_bias, activation="linear", name=f"dense_{i}")(x)

        if use_batch_normalization:
            x = keras.layers.BatchNormalization(name=f"batchnorm_{i}")(x)

        x = keras.layers.ReLU(name=f"relu_{i}")(x)
        
        if use_dropout:
            x = keras.layers.Dropout(0.5, name=f"dropout_{i}")(x)
    
    output_x = keras.layers.Dense(1, name="vel_x")(x)
    output_y = keras.layers.Dense(1, name="vel_y")(x)

    outputs = {
        "vel_x": output_x,
        "vel_y": output_y,
    }

    return keras.models.Model(
        inputs=inputs,
        outputs=outputs,
        name=model_description.get("name", backbone_name)
    )

# Results structure

In [14]:
results = {}

for model_description in models_descriptions:
    model_name = model_description["name"]
    
    results[model_name] = []

In [15]:
def store_predictions_h5(model, test_ds, file_path: str):
    all_true_vel_x = []
    all_pred_vel_x = []
    all_true_vel_y = []
    all_pred_vel_y = []

    for images, labels in test_ds:
        preds = model.predict(images, verbose=0)
        
        if isinstance(preds, (list, tuple)):
            pred_vel_x, pred_vel_y = preds
        elif isinstance(preds, dict):
            pred_vel_x, pred_vel_y = preds["vel_x"], preds["vel_y"]
        else:
            raise ValueError("Formato de saída inesperado de model.predict.")

        all_true_vel_x.append(labels["vel_x"].numpy())
        all_pred_vel_x.append(pred_vel_x)

        all_true_vel_y.append(labels["vel_y"].numpy())
        all_pred_vel_y.append(pred_vel_y)

    all_true_vel_x = np.concatenate(all_true_vel_x, axis=0)
    all_pred_vel_x = np.concatenate(all_pred_vel_x, axis=0)

    all_true_vel_y = np.concatenate(all_true_vel_y, axis=0)
    all_pred_vel_y = np.concatenate(all_pred_vel_y, axis=0)

    with h5py.File(file_path, "w") as f:
        f.create_dataset("true_vel_x", data=all_true_vel_x)
        f.create_dataset("pred_vel_x", data=all_pred_vel_x)
        f.create_dataset("true_vel_y", data=all_true_vel_y)
        f.create_dataset("pred_vel_y", data=all_pred_vel_y)

    print(f"[INFO] True and predicted labels saved in {file_path}")

# Benchmark loop

In [16]:
samples_dataframe = load_samples_dataframe("/kaggle/input/marketplace-navigation-dataset/dataset")

seed = 3

datasets = {}

for model_description in models_descriptions:
    dataset_key = tuple(model_description["input"]["shape"])
    dataset_key = model_description["input"]["grayscale"], dataset_key
    
    if not dataset_key in datasets:
        datasets[dataset_key] = build_train_val_test_datasets(
            samples_dataframe,
            model_description["input"],
            seed
        )
        
for learning_rate, batch_size in configurations:
    for model_description in models_descriptions:
        print(
            "Training model", model_description["name"],
            "for configuration LR =", learning_rate, ", BS =", batch_size 
        )
        
        dataset_key = tuple(model_description["input"]["shape"])
        dataset_key = model_description["input"]["grayscale"], dataset_key

        train_ds = datasets[dataset_key]["train"]
        val_ds   = datasets[dataset_key]["val"]
        test_ds  = datasets[dataset_key]["test"]

        gen = tf.random.Generator.from_seed(seed)
        augment = get_augment_function(gen)

        train_ds = train_ds.shuffle(train_ds.cardinality(), seed=seed, reshuffle_each_iteration=True)
        train_ds = train_ds.map(augment, num_parallel_calls=tf.data.AUTOTUNE)
        
        train_ds = train_ds.batch(batch_size, drop_remainder=True).prefetch(tf.data.AUTOTUNE)
        val_ds   = val_ds.batch(batch_size).prefetch(tf.data.AUTOTUNE)
        test_ds  = test_ds.batch(batch_size).prefetch(tf.data.AUTOTUNE)

        model = build_model(model_description)
        model.compile(
            loss={
                "vel_x": "mean_squared_error",
                "vel_y": "mean_squared_error",
            },
            optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
            metrics={
                "vel_x": [rmse, eva, r2_score],
                "vel_y": [rmse, eva, r2_score],
            },
        )

        history_1 = model.fit(
            train_ds,
            validation_data=val_ds,
            epochs=200,
            verbose=0,
            callbacks=[
                keras.callbacks.EarlyStopping(
                    monitor="val_loss",
                    patience=15,
                    min_delta=1e-4,
                    mode="min",
                    restore_best_weights=True,
                ),
            ],
        )

        for layer in model.layers:
            if layer.name == model_description["backbone"]["name"]:
                layer.trainable = True
                print("[INFO] Unfrozen layer", layer.name)
        
        model.compile(
            loss={
                "vel_x": "mean_squared_error",
                "vel_y": "mean_squared_error",
            },
            optimizer=keras.optimizers.Adam(learning_rate=learning_rate * 0.1),
            metrics={
                "vel_x": [rmse, eva, r2_score],
                "vel_y": [rmse, eva, r2_score],
            },
        )

        history_2 = model.fit(
            train_ds,
            validation_data=val_ds,
            verbose=0,
            epochs=200,
            callbacks=[
                keras.callbacks.EarlyStopping(
                    monitor="val_loss",
                    patience=15,
                    min_delta=1e-4,
                    mode="min",
                    restore_best_weights=True,
                ),
                keras.callbacks.ReduceLROnPlateau(
                    monitor="val_loss",
                    patience=5,
                    factor=0.5,
                    min_lr=1e-7,
                    mode="min"
                ),
            ],
        )
    
        history = pd.concat([
            pd.DataFrame(history_1.history),
            pd.DataFrame(history_2.history),
        ]).reset_index(drop=True)

        metrics = model.evaluate(test_ds, return_dict=True, verbose=0)
        
        results[model_description["name"]].append({
            "learning rate": learning_rate,
            "batch size": batch_size,
            "metrics": metrics,
        })

        file_name = f"{model.name}_{learning_rate:.0e}_{batch_size}"

        model.save(f"{file_name}.keras")
        history.to_csv(f"{file_name}.csv")
        store_predictions_h5(model, test_ds, f"{file_name}.h5")
        
        with open("results.json", "w") as results_file:
            results_file.write(json.dumps(results, indent=True))

        del model
        keras.backend.clear_session()

I0000 00:00:1752665333.532236      19 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


Training model ResNet-8-base for configuration LR = 0.01 , BS = 32


I0000 00:00:1752665356.546254      58 service.cc:148] XLA service 0x7b4ac40021c0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1752665356.547216      58 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1752665356.901820      58 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1752665358.876558      58 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


[INFO] Unfrozen layer ResNet-8
[INFO] True and predicted labels saved in ResNet-8-base_1e-02_32.h5
Training model ResNet-8-d32 for configuration LR = 0.01 , BS = 32
[INFO] Unfrozen layer ResNet-8
[INFO] True and predicted labels saved in ResNet-8-d32_1e-02_32.h5
Training model ResNet-8-d64 for configuration LR = 0.01 , BS = 32
[INFO] Unfrozen layer ResNet-8
[INFO] True and predicted labels saved in ResNet-8-d64_1e-02_32.h5
Training model ResNet-8-d128 for configuration LR = 0.01 , BS = 32
[INFO] Unfrozen layer ResNet-8
[INFO] True and predicted labels saved in ResNet-8-d128_1e-02_32.h5
Training model MobileNet for configuration LR = 0.01 , BS = 32
17225924/17225924 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
[INFO] Unfrozen layer MobileNet


E0000 00:00:1752666099.890205      59 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1752666100.090543      59 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.


...

[INFO] True and predicted labels saved in EfficientNetB0_1e-02_32.h5
Training model ResNet-50 for configuration LR = 0.01 , BS = 32
94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step
[INFO] Unfrozen layer ResNet-50
[INFO] True and predicted labels saved in ResNet-50_1e-02_32.h5
Training model ResNet-50V2 for configuration LR = 0.01 , BS = 32
94668760/94668760 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step
[INFO] Unfrozen layer ResNet-50V2
[INFO] True and predicted labels saved in ResNet-50V2_1e-02_32.h5
Training model ResNet-8-base for configuration LR = 0.01 , BS = 64
[INFO] Unfrozen layer ResNet-8
[INFO] True and predicted labels saved in ResNet-8-base_1e-02_64.h5
Training model ResNet-8-d32 for configuration LR = 0.01 , BS = 64
[INFO] Unfrozen layer ResNet-8
[INFO] True and predicted labels saved in ResNet-8-d32_1e-02_64.h5
Training model ResNet-8-d64 for configuration LR = 0.01 , BS = 64
[INFO] Unfrozen layer ResNet-8
[INFO] True and predicted labels saved in ResNet-8-d64_1e-02_64.h5
Training mo

E0000 00:00:1752668834.134315      57 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1752668834.336317      57 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.


[INFO] True and predicted labels saved in MobileNet_1e-02_64.h5
Training model EfficientNetB0 for configuration LR = 0.01 , BS = 64
[INFO] Unfrozen layer EfficientNetB0


E0000 00:00:1752669492.059140      56 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1752669492.253057      56 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1752669492.810839      56 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1752669493.019334      56 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1752669493.413081      56 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:0

[INFO] True and predicted labels saved in EfficientNetB0_1e-02_64.h5
Training model ResNet-50 for configuration LR = 0.01 , BS = 64
[INFO] Unfrozen layer ResNet-50
[INFO] True and predicted labels saved in ResNet-50_1e-02_64.h5
Training model ResNet-50V2 for configuration LR = 0.01 , BS = 64
[INFO] Unfrozen layer ResNet-50V2
[INFO] True and predicted labels saved in ResNet-50V2_1e-02_64.h5
Training model ResNet-8-base for configuration LR = 0.001 , BS = 32
[INFO] Unfrozen layer ResNet-8
[INFO] True and predicted labels saved in ResNet-8-base_1e-03_32.h5
Training model ResNet-8-d32 for configuration LR = 0.001 , BS = 32
[INFO] Unfrozen layer ResNet-8
[INFO] True and predicted labels saved in ResNet-8-d32_1e-03_32.h5
Training model ResNet-8-d64 for configuration LR = 0.001 , BS = 32
[INFO] Unfrozen layer ResNet-8
[INFO] True and predicted labels saved in ResNet-8-d64_1e-03_32.h5
Training model ResNet-8-d128 for configuration LR = 0.001 , BS = 32
[INFO] Unfrozen layer ResNet-8
[INFO] True